In [1]:
import numpy as np

In [2]:
import pandas as pd

# Load The Data Set Movies & Credits

In [4]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

# Explore The Data Set

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

As wee see in the movies dataset there is all the important information about movies like when movies release and in which language also about generes and budgets of the movies so we will keep some of these columns and skip some columns that we dont need.

.................

Similarly, in the credits dataset we have movies_id and information about cast of the movie and crew (all the other members in the movie) So, we have to datasets in the first step we need to be merged them so that we cxan easily handle only one dataframe for project, in the below stpe we will merge these two datasets.

# We are merging these two datasets on the basis of titles

In [ ]:
movies = movies.merge(credits, on="title")

In [ ]:
movies.shape

#Now after merging the datasets we have 23 columns and 4809 rows

It seems that after merging the dataset we will have 24 columns, but we find it 23 This is because we meerged it on the basis odf title so it is not in the counting 

In [ ]:
movies.head()

# Now lets Find out the columns that we need to be used 

As we know we have to make a content based recommendation system so we are trying to create tags.

here's the columns that are important and we will consider for our project

1. genres
2. movie_id
3. keywords
4. title
5. overview
6. cast (based on character like Salman khan etc)
7. crew (director etc)

now we will extract these columns for further proceed

In [ ]:
movies = movies[["movie_id","title","overview","genres","keywords", "cast","crew"]]

In [ ]:
movies.head()

now we will create a new column TAGS, and we will combine overivew, generes, keywords, cast and crew to get tags column in this way we will have 3 columns 

1. movie_id
2. title
3. tags

# Data Preprocessing 

In [ ]:
# handle missing values

movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
#Checking the duplicate data

movies.duplicated().sum()

In [ ]:
# chek the genere columns for merging to get tag colum

movies.iloc[0].genres

This is the list of dictionaries and our task is to get just this list 

["Action","Adventure","Fantasy","Science Fiction"]

for this will make a function that will get this work done 

In [ ]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):        
        L.append(i["name"])
    return L

In [ ]:
movies["genres"].apply(convert)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

In [ ]:
# same for keyword colums

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L 

In [ ]:
movies["cast"] = movies["cast"].apply(convert3)

In [ ]:
movies.head()

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies["overview"] = movies["overview"].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
#remove the spaces in double words (one name) from these columns for avoiding conflit to get recomendation , becasue first names may be same for 2 persons 

movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ","") for i in x])
movies["keywords"] = movies["keywords"].apply(lambda x:[i.replace(" ","") for i in x])
movies["cast"] = movies["cast"].apply(lambda x:[i.replace(" ","") for i in x])
movies["crew"] = movies["crew"].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

# Concatenate the above columns to get tag column

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

As we acheived our task to make tag column now the other columns we will remove 

In [ ]:
new_df = movies[["movie_id", "title", "tags"]]

In [ ]:
new_df

In [ ]:
#now convert the list of tags column(values) into string

new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df["tags"][0]

In [ ]:
new_df["tags"] = new_df["tags"].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df["tags"] = new_df["tags"].apply(stem)

# Now Text vectorization

our task is to make a system that recommend the same type of movies based on tags, 

so our 1st task is to find out the similarity between the movies tags (paragraph) thats why we will use vectorization, we will apply technique bags of words on the tags columns so that we will have vector coresponding to each move and in this wahy we can get our target similar words for recommendation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
       
    
    

In [ ]:
recommend("Avatar")

In [5]:
import pickle

In [6]:
pickle.dump(new_df.to_dict(), open("movie_dict.pkl","wb"))

NameError: name 'new_df' is not defined

In [ ]:
pickle.dump(similarity, open("similarity.pkl","wb"))